In [1]:
import sys
import os
import numpy as np
import torch
import random

# 添加环境
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../MyExpr")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../FedML")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))

print(sys.path)

['/home/ubuntu/Fed_Expr', '/home/ubuntu/Fed_Expr/FedML', '/home/ubuntu/Fed_Expr/MyExpr', '/home/ubuntu/Fed_Expr/MyExpr/ml', '/home/ubuntu/miniconda/envs/fedml/lib/python37.zip', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/lib-dynload', '', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [2]:
import torch

# logistic
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [3]:
import torch.nn as nn


# todo neural network
# 5优化神经网络
# 5.1模型改进
class Activation_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2,n_hidden_3, out_dim):
        super(Activation_Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, n_hidden_3), nn.ReLU(True))
        """
        请补充 利用nn.Sequential填写最后一层代码
        """
        self.layer4 = nn.Linear(n_hidden_3, out_dim)
        # nn.Sequential(nn.Linear(n_hidden_3, out_dim), nn.Softmax())
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

# 检查模型是否正确构建
model = Activation_Net(18, 100, 50, 30, 1)
print(model)

In [4]:
import argparse

parser = argparse.ArgumentParser("cifar")
parser.add_argument('--data_name', type=str, default="SUSY", help='SUSY; RO')
parser.add_argument('--epoch_size', type=int, default=500, help='1,2,3,4,5')
parser.add_argument('--learning_rate', type=float, default=0.1)
parser.add_argument("--batch_size", type=int, default=1000)
parser.add_argument('--weight_decay', type=float, default=0.0001)

# 报错，因为Jupyter会读入一个默认的环境参数-f
# parser.parse_args()
args = parser.parse_known_args()[0]

# wandb
import wandb

wandb.init(project="susy_nonfl",
            entity="kyriegyj",
           config=args)

# fix random seeds
seed = 127834698
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

/home/ubuntu/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/ubuntu/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/ubuntu/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/ubuntu/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
Failed to detect the name of this notebook, you 

In [ ]:
# load data
from MyExpr.ml.data_loader_for_susy import DataLoader

epoch_size = args.epoch_size
batch_size = args.batch_size

reload = False

data_loader = DataLoader("../../FedML/data/UCI/SUSY/SUSY.csv", batch_size = batch_size)

if reload:
    train_X, train_Y, test_X, test_Y = data_loader.load_data()
# print(len(train_X), len(test_X), batch_size, "5,000,000")

In [ ]:
import math

learning_rate = args.learning_rate
weight_decay = args.weight_decay

model = LogisticRegression(18, 1)
iteration = len(train_X)
# print(iteration, batch_size)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# print(optimizer)
# print(model)

test = False
for e in range(epoch_size):
    # train
    correct = 0
    total_loss = 0
    for i in range(iteration):
        x, y = torch.tensor(np.asarray(train_X[i], dtype=np.float32), dtype=torch.float32), torch.tensor(np.asarray(train_Y[i], dtype=np.float32), dtype=torch.float32)
       
        # print(x)
        # mean_x = torch.mean(x, dim = 1)
        # std_x = x.std(dim = 1)
        # # print(mean_x.shape, std_x.shape)
        # x = (x - mean_x.unsqueeze(1).expand(x.shape[0], x.shape[1])) / std_x.unsqueeze(1).expand(x.shape[0], x.shape[1])
        # print(x)
        
        # [batch_size, 1]
        output = model(x)
        # print(output)
        # print(x, y)
        # print(type(x), type(y))
        # print(x.shape, y.shape, output.shape)
        
        # 1 dimension
        loss = criterion(output.squeeze(1), y)
        
        # 求第一维的最大值以及对应下标，应该是配合softmax使用(取预测概率最大的类下标作为预测结果，但是二分类只有一个输出。。)
        # pred[0]为最大值，pred[1]为最大值的下标
        # pred = output.max(1, keepdim = True)[1]
        
        # 求第一维的最大值对应的下标
        # pred = torch.argmax(output, 1);
        pred = output.ge(0.5).float()
        # print(pred)
        # print(pred.shape, y.shape)
        # print(output.shape, loss, pred[0].shape, pred[1].shape, len(pred))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        correct += (pred.squeeze(1) == y).sum().item()
        total_loss += loss.detach().numpy()
        # print(correct, pred.shape)
        if test: break
    if test: break
    
    print("train_loss:", total_loss, "train_acc:", float(correct / (iteration * batch_size)), "epoch:", e)
    if not test:
        wandb.log({"train_loss": total_loss, "train_acc": float(correct / (iteration * batch_size)), "epoch": e})
    

if not test:
    log_file_path = "log/wandb.log"
    wandb.save(log_file_path)
    # if test:
    #     break
       
        # output =
    # test

4500 1000
train_loss: 2132.747630894184 acc: 0.7832413333333333 epoch: 0
train_loss: 2079.321749597788 acc: 0.78822 epoch: 1
train_loss: 2074.062014937401 acc: 0.7884904444444445 epoch: 2
train_loss: 2071.4913159012794 acc: 0.7885322222222222 epoch: 3
train_loss: 2069.858170300722 acc: 0.7885693333333333 epoch: 4
train_loss: 2069.058108061552 acc: 0.7885251111111111 epoch: 5
train_loss: 2068.5616105794907 acc: 0.7884724444444444 epoch: 6
train_loss: 2068.3146290779114 acc: 0.7884655555555555 epoch: 7
train_loss: 2068.159271001816 acc: 0.788436 epoch: 8
train_loss: 2068.0593761503696 acc: 0.7884353333333334 epoch: 9
train_loss: 2067.992193311453 acc: 0.7884317777777777 epoch: 10
train_loss: 2067.9471566081047 acc: 0.7884268888888889 epoch: 11
train_loss: 2067.9161597788334 acc: 0.7884304444444444 epoch: 12
train_loss: 2067.8941952586174 acc: 0.7884302222222223 epoch: 13
train_loss: 2067.8784460127354 acc: 0.7884302222222223 epoch: 14
train_loss: 2067.7831463217735 acc: 0.788428 epoch: 1

In [ ]:

test_acc = 0
test_loss = 0
with torch.no_grad():
    for i in range(len(test_X)):
        x, y = torch.tensor(np.asarray(test_X[i], dtype=np.float32), dtype=torch.float32), torch.tensor(np.asarray(test_Y[i], dtype=np.float32), dtype=torch.float32)
        output = model(x)
        loss = criterion(output.squeeze(1), y)
        pred = output.ge(0.5).float()
        correct += (pred.squeeze(1) == y).sum().item()
        total_loss += loss.detach().numpy()
    print("test_loss:", total_loss, "test_acc:", float(correct / (len(test_X) * batch_size)))

In [ ]:
# a = torch.tensor(np.asarray([[1.1], [0.7], [0.9]], dtype=np.float32), dtype=torch.float32)
# print(a)
# print(a.max(1, keepdim = True))

# a = torch.rand(3, 1)
# print(a)
# print(a.max(1, keepdim = True))

# a = torch.randn(10, 2)
# b = torch.randn(10)

# # print(a.shape)
# # print(b.shape)
# print(b)
# print(b.unsqueeze(1).expand(10,2))

a = torch.tensor(np.asarray([[0.1], [0.7], [0.9]], dtype=np.float32), dtype=torch.float32)
print(a.shape)
print(a[a>0.5].shape[0])
print(a.ge(0.5).float())

print(torch.tensor([True, False]).sum())
